# Ensemble di MLPClassifier con Soft Voting
In questo notebook si utilizza una tecnica ensemble per migliorare la robustezza dfel modello. Vengono addestrati più MLPClassifier con architettura identica e random state diversi e si aggregano le predizioni tramite media delle probabilità.


In [1]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import os
from imblearn.over_sampling import SMOTE
import sys
sys.path.append('../Scripts')
from utility import evaluate_and_save_model_multiclass

X_train = pd.read_csv("../data/splitted_category/X_train.csv")
X_test = pd.read_csv("../data/splitted_category/X_test.csv")
y_train = pd.read_csv("../data/splitted_category/y_train.csv").values.ravel()
y_test = pd.read_csv("../data/splitted_category/y_test.csv").values.ravel()  

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



seeds = [1, 42, 99]
mlps = []

for seed in seeds:
    mlp = MLPClassifier(
        hidden_layer_sizes=(64, 32),
        activation='relu',
        solver='adam',
        alpha=0.05,
        learning_rate='adaptive',
        max_iter=300,
        early_stopping=True,
        validation_fraction=0.2,
        random_state=seed
    )
    mlp.fit(X_train_scaled, y_train)
    mlps.append(mlp)


# === Media delle probabilità (Soft Voting) TRAIN 
probas_train = np.array([model.predict_proba(X_train_scaled) for model in mlps])
avg_probas_train = np.mean(probas_train, axis=0)
y_pred_train_ensemble = np.argmax(avg_probas_train, axis=1)

# === Media delle probabilità (Soft Voting) TEST
probas = np.array([model.predict_proba(X_test_scaled) for model in mlps])
avg_probas = np.mean(probas, axis=0)
y_pred_ensemble = np.argmax(avg_probas, axis=1)

evaluate_and_save_model_multiclass(
    mlp,
    "mlps e media delle probabilità",
    y_train,
    y_pred_train_ensemble,
    y_test,
    y_pred_ensemble,
    "../results/classification_category/ann",
    "../models/ann_ensemble.joblib",
    {"hidden layer":(64,32),"activation":"relu","solver":"adam","learning rate":"adaptive","alpha":0.05,"media modelli":"3 modelli mlp"}
)
